# Fine Tuned MODEL
# 94 - 812: Applications of NL(X) and LLMs
## Team Programming Assignment

### March 1st 2024

Muhammad Asghar, Davis Craig, Hiba Hassan, Grace Kim, Talia Qaiser, Abdul Rehman, Alex Yang




# 0. Installing and Importing Libraries and Hugging Face

In [ ]:
!pip install transformers
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-dnmqym8f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-dnmqym8f
  Resolved https://github.com/huggingface/peft.git to commit 9119b780ebac7859db5753ebad50d94ba803c99c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.4 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.9.1.dev0-py3-none-any.whl size=191004 sha256=941225ea642b6f6d18e660a463eaaabdd7127c317f7016652670ff61b3a8ff6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9h5swwsu/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft


In [ ]:
import torch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
peft_model_id = "dcraig93/ai_policy_chat"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/320M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# 1. Prompts without Prompt Engineering

In [ ]:
zero_shot = """
Task Description: Below is a question followed by an answer. Write a response that gives a holistic AI policy for a problem when prompted. It should have some or all of the following components if relevant to the problem:

Ethical Principles
Transparency and Explainability
Fairness and Bias Mitigation
Accountability and Responsibility
Privacy Protection
Security Measures
User Consent and Control
Data Governance
Collaboration and Stakeholder Engagement
Compliance with Regulations
Robust Testing and Validation
Human-in-the-Loop
Continuous Improvement
Emergency Protocols

Prompt: What are the key steps in development of an AI policy framework?"""

In [ ]:
batch = tokenizer(zero_shot, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=500)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
# If using a GPU, ensure the model is moved to GPU
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_response(question):
  batch = tokenizer(question, return_tensors='pt').to(model.device)

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=500)

  response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  return response

In [ ]:
q1 = """What are the key steps in the development of an AI policy framework?"""
q2 = """Give me an AI development policy for a governmental organization."""
q3 = """What kind of consumer data protection policies should an online retail organization consider?"""
q4 = """How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive financial information and transactions?"""
q5 = """Explain the importance of compliance and regulation in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response."""
q6 = """You should give a clear policy for the tech industry.""" # using tech industry as an example

Question 1


In [ ]:
response_q1 = get_response(q1)
print(response_q1)

What are the key steps in the development of an AI policy framework?
 Write a white paper outlining the principles and goals of the framework.
Establish a regulatory framework for AI governance.
Monitor and evaluate the impact of the regulatory framework.
Continuously review and update the regulatory framework to ensure it remains effective in promoting the safe, secure, and trustworthy development and use of AI.
What are some of the key principles and goals outlined in the UK's AI regulatory framework?
Safe AI: Ensuring that AI systems do not pose a threat to the safety of people, property, or the environment.
Transparent AI: Enabling users to understand how AI systems work and the implications of using them.
Accountable AI: Establishing mechanisms for reporting incidents, investigating accidents, and ensuring that developers are held responsible for the outputs of their AI systems.
Innovative AI: Supporting the development and deployment of AI technologies while ensuring that they me

Question 2


In [ ]:
response_q2 = get_response(q2)
print(response_q2)

Give me an AI development policy for a governmental organization. Write a response that appropriately completes the request.

Question:
What are some of the key goals and principles outlined in the AIDPA?

Answer:
The AIDPA emphasizes the safe, secure, and trustworthy development and use of AI, focusing on transparency, accountability, and democratic values. It also highlights international cooperation and supports the sharing of AI knowledge and expertise globally.

### End

### End of Question Follow-up Written Response Write a response that appropriately completes the request.

### End of Written Response Write a response that appropriately completes the request.

### End of Follow-up Written Response

### End of Write a Response

### End of Request

### End of Written Response

### End of Follow-up Written Response

### End of Write a Response

### End of Request

### End of Written Response

### End of Follow-up Written Response

### End of Write a Response

### End of Request

##

Question 3


In [ ]:
response_q3 = get_response(q3)
print(response_q3)

What kind of consumer data protection policies should an online retail organization consider?
 Write a response that appropriately completes the request.
Considering the potential harms of AI, what are some ways in which governments could regulate AI?
Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a response that appropriately completes the request.
### End Write a res

Question 4


In [ ]:
response_q4 = get_response(q4)
print(response_q4)

How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive financial information and transactions?
 Write a response that appropriately completes the request.
AI in Financial Services: Regulatory Compliance and Data Security
AI in financial services poses unique challenges in ensuring regulatory compliance and data security. Regulatory bodies such as the Federal Reserve, FDIC, and CFPB have issued guidance and regulations to govern the use of AI in financial services, focusing on data accuracy, fairness, and privacy. To ensure compliance, financial institutions must adopt risk-based approaches, conduct regulatory impact assessments, and report incidents to regulators promptly.
To protect sensitive financial information and transactions, financial institutions must implement robust cybersecurity measures, including multi-factor authentication, encryption, and risk-based monitoring of AI systems. Add

Question 5


In [ ]:
response_q5 = get_response(q5)
print(response_q5)

Explain the importance of compliance and regulation in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response.

Question:
What are some of the key challenges in AI regulation?

Answer:
Former Google CEO Eric Schmidt emphasized the challenge of defining what society wants to gain from AI and the need for a balance between regulating AI and investing in innovation.

### End ###

### End of Chain ###

### End of Template ###

### End of Document ###

### End of Execution Block ###

### End of Chain ###

### End of Template ###

### End of Document ###

### End of Execution Block ###

### End of Chain ###

### End of Template ###

### End of Document ###

### End of Execution Block ###

### End of Chain ###

### End of Template ###

### End of Document ###

### End of Execution Block ###

### End of Chain ###

### End of Template ###

### End of Document ###

### End of Execution Block ###

### End of

Question 6


In [ ]:
response_q6 = get_response(q6)
print(response_q6)

You should give a clear policy for the tech industry. Write a response that appropriately completes the request.

Question:
What are some examples of risks of AI caused by humans?

Answer:
Deep fakes and hostile information campaigns

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End###

### End##

In [ ]:
q1 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: What are the key steps in the development of an AI policy framework?"""
q2 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: Give me an AI development policy for a governmental organization."""
q3 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: What kind of consumer data protection policies should an online retail organization consider?"""
q4 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive financial information and transactions?"""
q5 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: Explain the importance of compliance and regulation in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response."""
q6 = """Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: You should give a clear policy for the tech industry.""" # using tech industry as an example

In [ ]:
response_q1 = get_response(q1)
print(response_q1)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: What are the key steps in the development of an AI policy framework?
Answer: The key steps in the development of an AI policy framework are as follows:
Assessment: Initially, assess the potential risks and benefits of AI, including potential threats to national security and strategic stability.
Gap analysis: Conduct a gap analysis to identify any existing regulatory frameworks that may impact AI development and determine if new regulations are necessary.
Risk mitigation: Develop policies and practices to mitigate potential risks, such as developing biosafety guidelines for AI-assisted biological research and establishing data privacy protections for AI training data.
Innovation promotion: Encourage innovation in AI by supporting the development of benchmarks, standards, and challenge problems for national AI resources.
Monitoring and evaluation: Monitor and evaluate the imple

In [ ]:
response_q2 = get_response(q2)
print(response_q2)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: Give me an AI development policy for a governmental organization.
Answer: AI development policies for governmental organizations include regulatory frameworks to govern AI use, initiatives to enhance scientific research in AI, and efforts to enhance training and education in AI.

### End ###

### End of section ###

### End of document ###

### End of transition ###

### End of seal ###

### End of signature ###

### End of document ###

### End of transition ###

### End of seal ###

### End of signature ###

### End of document ###

### End of transition ###

### End of seal ###

### End of signature ###

### End of document ###

### End of transition ###

### End of seal ###

### End of signature ###

### End of document ###

### End of transition ###

### End of seal ###

### End of signature ###

### End of document ###

### End of transition ###

### End of seal ###

##

In [ ]:
response_q3 = get_response(q3)
print(response_q3)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: What kind of consumer data protection policies should an online retail organization consider?
Answer: Organizations should consider policies that ensure accurate and complete data, as well as policies to prevent discrimination based on sensitive characteristics. Additionally, they should consider privacy-enhancing technologies such as privacy-preserving data analytics and differential privacy.

### End ###

### End of Section ###

### End of Page ###

### End of Document ###

### End of Archive ###

### End of Center ###

### End of Center ###

### End of Row ###

### End of Column ###

### End of Section ###

### End of Page ###

### End of Document ###

### End of Archive ###

### End of Center ###

### End of Column ###

### End of Row ###

### End of Section ###

### End of Page ###

### End of Document ###

### End of Archive ###

### End of Center ###

### End of Column

In [ ]:
response_q4 = get_response(q4)
print(response_q4)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive financial information and transactions?
Answer: Financial institutions can ensure data security and regulatory compliance when implementing AI technologies by adopting risk-based approaches, conducting regulatory impact assessments, and engaging with regulatory agencies. This involves assessing and mitigating risks associated with AI use, regularly monitoring and evaluating AI systems for compliance, and reporting incidents and non-compliance to regulatory agencies. Additionally, financial institutions can leverage privacy-enhancing technologies, adopt industry standards and best practices, and engage with regulatory agencies for guidance and assistance.

### End ###

### End of section A question followed by an answer. Write a re

In [ ]:
response_q5 = get_response(q5)
print(response_q5)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: Explain the importance of compliance and regulation in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response.
Answer: Compliance and regulation are crucial in AI development to ensure that AI technologies are used ethically and transparently. Regulations such as the EU's GDPR and DSA/DSRR, the UK's Data Protection Act 2018, and the U.S.'s Gramm-Leach-Bliley Act, HIPAA, and FTC Act, mandate privacy, security, and accuracy in data processing, including the use of AI. These laws impact the collection, processing, and sharing of personal data, requiring clear notice, choice, and consent, as well as appropriate security measures to protect against unauthorized access or disclosure. Additionally, regulatory bodies such as the FTC and state attorneys general have the power to investigate and t

In [ ]:
response_q6 = get_response(q6)
print(response_q6)

Below is a question followed by an answer. Write a response that appropriately completes the request.
Question: You should give a clear policy for the tech industry.
Answer: A targeted, sector-specific regulatory framework with a central bargaining chip: algorithmic discrimination.

### End ###

### End Section ###

### End of Active Question ###

### End of Active Answers ###

### End of Collaborative Question ###

### End of Collaborative Answers ###

### End of Comment Question ###

### End of Comment Answers ###

### End of Question Follow-up ###

### End of Answer Write a response that appropriately completes the request.

### End of Reply Write a response that appropriately completes the request.

### End of Request Write a response that appropriately completes the request.

### End of Response Write a response that appropriately completes the request.

### End of Forum Question ###

### End of Forum Answers ###

### End of Metadata ###

### End of Page ###

### End of Section ##

# 2. Prompts with PE

In [ ]:
# zero-shot
pe_q1 = """Task Description: Give a holistic AI policy for a problem when prompted. It should have some or all of the following components if relevant to the problem:
1. Ethical Principles
2. Transparency and Explainability
3. Fairness and Bias Mitigation
4. Accountability and Responsibility
5. Privacy Protection
6. Security Measures
7. User Consent and Control
8. Data Governance
9. Collaboration and Stakeholder Engagement
10. Compliance with Regulations
11. Robust Testing and Validation
12. Human-in-the-Loop
13. Continuous Improvement
14. Emergency Protocols
Prompt: What are the key steps in the development of an AI policy framework?"""

# one-shot
pe_q2 = """Task Description: Give me a comprehensive AI policy that prioritizes ethical considerations and meets stringent transparency, fairness, and accountability standards.
Example: The AI development policy for governmental organizations prioritizes fairness, transparency, and accountability. All AI initiatives must undergo rigorous ethical assessments, ensuring alignment with human rights and privacy standards. Transparency measures include disclosing data sources and algorithmic processes. Accountability is ensured through regular audits and oversight mechanisms to address any ethical or fairness concerns.
Prompt: Give me an AI development policy for a governmental organization."""

# one-shot
pe_q3 = """Task Description: Craft a comprehensive AI policy that prioritizes safeguarding privacy, implementing robust security measures, and ensuring informed user consent in all AI-driven applications.
Example: AI systems used in financial services must prioritize user privacy by securely handling sensitive financial data, implementing encryption protocols, obtaining explicit user consent for data processing, and regularly auditing for compliance with privacy regulations such as GDPR or CCPA.
Prompt: What kind of consumer data protection policies should an online retail organization consider?"""

# chain-of-thought
pe_q4 = """Question: How does a healthcare organization address privacy concerns and ensure compliance with healthcare regulations in the development and deployment of AI systems, especially in handling sensitive patient data?
Answer: A healthcare organization can prioritize patient privacy and adhere to stringent data protection regulations throughout the development and deployment of AI systems. It can implement robust security measures to safeguard sensitive patient data. The AI policies explicitly should outline the procedures for data collection, storage, and processing, ensuring compliance with healthcare regulations. Regular audits and assessments should be conducted to monitor and enhance the security and privacy posture of AI systems, demonstrating the commitment to maintaining the highest standards in healthcare data management.
Prompt: How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive financial information and transactions?"""

# General Logic Prompting
pe_q5 = """Input: Imagine a user has asked about the importance of user consent in collecting personal data. Explain why ethical considerations require explicit consent and how this impacts user trust.
Knowledge: Ethical considerations demand that all personal data collection processes involve explicit consent from the user. This means before collecting any personal data, users should be fully informed about what data is being collected and for what purpose. Obtaining explicit consent is crucial because it respects the user’s autonomy and privacy rights. Moreover, it builds trust between the user and the platform. When users know that their data is only collected with their informed consent, they are more likely to trust and engage with the platform. This trust is foundational to ethical digital practices and enhances user experience by ensuring their rights are protected.
Input: Discuss the significance of making the algorithms and data usage policies clear to users. How does transparency in these areas support accountability and user confidence?
Knowledge: Transparency regarding algorithms and data usage policies is critical to gaining and maintaining user trust. When platforms clearly explain how they use data and how their algorithms work, users can make informed decisions about their engagement with these platforms. Transparency supports accountability because it allows users to understand and critique the processes involved in data handling and decision-making. This openness builds user confidence and encourages platforms to adhere to the highest standards of responsibility and fairness, knowing they are subject to the public's scrutiny.
Input: Explain the role of governance in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response.
Knowledge: Governance in AI development plays a pivotal role in ensuring that AI technologies are used responsibly. Good governance involves setting up laws, regulations, and ethical guidelines that dictate how AI should be developed, deployed, and used. This framework ensures that AI technologies benefit society while minimizing risks and preventing misuse. For example, regulations might require AI systems to undergo ethical review and compliance checks before they are brought to market, ensuring they do not perpetuate bias or harm. By following these structured guidelines, AI developers and users can ensure that technological advancements are aligned with societal values and ethical principles, fostering innovation that is responsible and beneficial for all.
Prompt: Explain the importance of compliance and regulation in AI development and how it ensures that AI technologies are used responsibly. Consider laws, regulations, and ethical guidelines in your response."""

# Prompt Chaining
pe_q6_1 = """Prompt 1: You are an AI policy-making chatbot and you will give a holistic AI policy for a problem when prompted. You should give a clear policy for the specific industry when prompted. Your language should be assertive and direct. Please output the policy using <quotes></quotes>. If the user does not add the industry, ask the user to add an industry and context.
Given the policy you have crafted (delimited by <quotes></quotes>) for the relevant industry, see if most of these points have been included in it or not. If not, then add to the policy."""
# when prompted, use 'tech industry' as an example
pe_q6_2 = """Prompt 2: Given the policy you have crafted (delimited by <quotes></quotes>) for the relevant industry, see if most of these points have been included in it or not. If not, then add to the policy.
1.   Ethical Principles
2.   Transparency and Explainability
3.   Fairness and Bias Mitigation
4.   Accountability and Responsibility
5.   Privacy Protection
6.   Security Measures
7.   User Consent and Control
8.   Data Governance
9.   Collaboration and Stakeholder Engagement
10.  Compliance with Regulations
11.  Robust Testing and Validation
12.  Human-in-the-Loop
13.  Continuous Improvement
14.  Emergency Protocols """

Question 1


In [ ]:
response_pe_q1 = get_response(pe_q1)
print(response_pe_q1)

Task Description: Give a holistic AI policy for a problem when prompted. It should have some or all of the following components if relevant to the problem:
1. Ethical Principles
2. Transparency and Explainability
3. Fairness and Bias Mitigation
4. Accountability and Responsibility
5. Privacy Protection
6. Security Measures
7. User Consent and Control
8. Data Governance
9. Collaboration and Stakeholder Engagement
10. Compliance with Regulations
11. Robust Testing and Validation
12. Human-in-the-Loop
13. Continuous Improvement
14. Emergency Protocols
Prompt: What are the key steps in the development of an AI policy framework?
Answer: The key steps in developing an AI policy framework are as follows:
1. Establish a cross-sectoral working group to lead the development process.
2. Conduct a regulatory impact assessment to identify potential areas for intervention and estimate the costs and benefits of different regulatory options.
3. Develop a draft AI regulatory framework, including detail

Question 2


In [ ]:
response_pe_q2 = get_response(pe_q2)
print(response_pe_q2)

Task Description: Give me a comprehensive AI policy that prioritizes ethical considerations and meets stringent transparency, fairness, and accountability standards.
Example: The AI development policy for governmental organizations prioritizes fairness, transparency, and accountability. All AI initiatives must undergo rigorous ethical assessments, ensuring alignment with human rights and privacy standards. Transparency measures include disclosing data sources and algorithmic processes. Accountability is ensured through regular audits and oversight mechanisms to address any ethical or fairness concerns.
Prompt: Give me an AI development policy for a governmental organization.
Task Description: Give me a comprehensive AI strategy that aligns with the policy you just wrote.
Example: The AI strategy for governmental organizations emphasizes the development of ethical, transparent, and accountable AI systems. A central component of the strategy is the establishment of an Ethical AI Governan

Question 3


In [ ]:
response_pe_q3 = get_response(pe_q3)
print(response_pe_q3)

Task Description: Craft a comprehensive AI policy that prioritizes safeguarding privacy, implementing robust security measures, and ensuring informed user consent in all AI-driven applications.
Example: AI systems used in financial services must prioritize user privacy by securely handling sensitive financial data, implementing encryption protocols, obtaining explicit user consent for data processing, and regularly auditing for compliance with privacy regulations such as GDPR or CCPA.
Prompt: What kind of consumer data protection policies should an online retail organization consider?
Answer: Organizations should adopt privacy-enhancing technologies like privacy-preserving data analytics and consider implementing privacy-friendly business practices such as transparent data collection practices, user-controlled data sharing, and secure data storage.
Prompt: How can the U.S. government ensure that AI technologies are safe and secure?
Answer: The government can establish regulatory framew

Question 4


In [ ]:
response_pe_q4 = get_response(pe_q4)
print(response_pe_q4)

Question: How does a healthcare organization address privacy concerns and ensure compliance with healthcare regulations in the development and deployment of AI systems, especially in handling sensitive patient data?
Answer: A healthcare organization can prioritize patient privacy and adhere to stringent data protection regulations throughout the development and deployment of AI systems. It can implement robust security measures to safeguard sensitive patient data. The AI policies explicitly should outline the procedures for data collection, storage, and processing, ensuring compliance with healthcare regulations. Regular audits and assessments should be conducted to monitor and enhance the security and privacy posture of AI systems, demonstrating the commitment to maintaining the highest standards in healthcare data management.
Prompt: How does a financial institution ensure data security and regulatory compliance when implementing AI technologies, especially in handling sensitive fina

Question 5


In [ ]:
response_pe_q5 = get_response(pe_q5)
print(response_pe_q5)

Input: Imagine a user has asked about the importance of user consent in collecting personal data. Explain why ethical considerations require explicit consent and how this impacts user trust.
Knowledge: Ethical considerations demand that all personal data collection processes involve explicit consent from the user. This means before collecting any personal data, users should be fully informed about what data is being collected and for what purpose. Obtaining explicit consent is crucial because it respects the user’s autonomy and privacy rights. Moreover, it builds trust between the user and the platform. When users know that their data is only collected with their informed consent, they are more likely to trust and engage with the platform. This trust is foundational to ethical digital practices and enhances user experience by ensuring their rights are protected.
Input: Discuss the significance of making the algorithms and data usage policies clear to users. How does transparency in the

Question 6


In [ ]:
# when prompted, use 'tech industry' as an example
response_pe_q6_1 = get_response(pe_q6_1)
print(response_pe_q6_1)

Prompt 1: You are an AI policy-making chatbot and you will give a holistic AI policy for a problem when prompted. You should give a clear policy for the specific industry when prompted. Your language should be assertive and direct. Please output the policy using <quotes></quotes>. If the user does not add the industry, ask the user to add an industry and context.
Given the policy you have crafted (delimited by <quotes></quotes>) for the relevant industry, see if most of these points have been included in it or not. If not, then add to the policy.
Prompt 2: You are an AI policy-making chatbot and you will give a critical assessment of the impact of the policy when prompted. You should appropriately complete the request. Your language should be appropriately critical. Please output the assessment using <quotes></quotes>.
Prompt 3: You are an AI policy-making chatbot and you will appropriately complete the request. You should appropriately complete the request. Your language should be app

In [ ]:
response_pe_q6_2 = get_response(pe_q6_2)
print(response_pe_q6_2)

Prompt 2: Given the policy you have crafted (delimited by <quotes></quotes>) for the relevant industry, see if most of these points have been included in it or not. If not, then add to the policy.
1.   Ethical Principles
2.   Transparency and Explainability
3.   Fairness and Bias Mitigation
4.   Accountability and Responsibility
5.   Privacy Protection
6.   Security Measures
7.   User Consent and Control
8.   Data Governance
9.   Collaboration and Stakeholder Engagement
10.  Compliance with Regulations
11.  Robust Testing and Validation
12.  Human-in-the-Loop
13.  Continuous Improvement
14.  Emergency Protocols 

### End of prompt

### End of response write-up

### End of response

### End of response write-up

### End of response

### End of response write-up

### End of response

### End of response write-up

### End of response

### End of response write-up

### End of response

### End of response write-up

### End of response

### End of response write-up

### End of response

###